In [1]:
import requests_html
import re
from tqdm.auto import tqdm
import pandas as pd
tqdm.pandas()
import io
from pprint import pprint

In [2]:
session = requests_html.HTMLSession()
with open("ci_session.cookie") as f:
  ci_session_cookie = f.read().strip()
_ = session.cookies.set("ci_session", ci_session_cookie)

In [3]:
results = set()
for page in tqdm(range(0, 516, 21)):
  resp = session.get(f"https://uoa.etv.org.nz/tv/vod_search?search=country+calendar&title_only=&per_page={page}")
  for link in resp.html.links:
    match = re.search(r"/tv/vod/view/(\d+)", link)
    if match:
      results.add(int(match.group(1)))
len(results)

  0%|          | 0/25 [00:00<?, ?it/s]

516

In [4]:
all_meta = []
for id in tqdm(results):
  html = io.StringIO(session.get(f"https://uoa.etv.org.nz//tv/vod/download_metadata/{id}").text)
  meta = pd.read_html(html)[0]
  meta["id"] = id
  all_meta.append(meta)
all_meta = pd.concat(all_meta)

  0%|          | 0/516 [00:00<?, ?it/s]

In [6]:
def get_link(id):
  resp = session.head(f"https://uoa.etv.org.nz/tv/vod/download_mp4/{id}")
  return resp.headers["Location"]

all_meta["link"] = all_meta["id"].progress_apply(get_link)

  0%|          | 0/516 [00:00<?, ?it/s]

In [18]:
def get_size(link):
  return session.head(link).headers["Content-Length"]

all_meta["size_bytes"] = all_meta["link"].progress_apply(get_size)

  0%|          | 0/516 [00:00<?, ?it/s]

In [19]:
all_meta.sort_values("Recording Date")

,File Name,Name,Description,Duration,Type,Subject,Age Level,Channel,Recording Date,Recording Time,id,link,size_bytes
0,etv/country_calendar_-_air_new_zealand_uniform...,country calendar - air new zealand uniforms 23...,from www.throng.co.nz : this episode of nation...,00:21:33,Library,NaN,G,lostplusfound,0000-00-00,00:00:00,61580,https://cdn.etv.org.nz/etv/country_calendar_-_...,145371146
0,etv/TV_One_1900_21022009_20090221193153256qtst...,Country Calendar - Series 2009 - Episode 1,'Lyn of Taranaki' Country Calendar starts its ...,00:29:57,Documentary,"Agriculture & Horticulture, Business, Ecology ...",Unrated,TV 1,2009-02-21,19:00:00,12564,https://cdn.etv.org.nz/etv/TV_One_1900_2102200...,49973516
0,etv/TV_One_1900_28022009_20090228193154256qtst...,Country Calendar - Series 2009 - Episode 2,'Shared Fruit' New Zealand and Chinese scienti...,00:29:56,Documentary,"Agriculture & Horticulture, Asian Studies, Bus...",Unrated,TV 1,2009-02-28,19:00:00,12697,https://cdn.etv.org.nz/etv/TV_One_1900_2802200...,52854708
0,etv/TV_One_1900_07032009_20090307193229256qtst...,Country Calendar - Series 2009 - Episode 3,'Grassroots Leader' Bruce Rollinson is a late-...,00:29:56,Documentary,"Agriculture & Horticulture, Business, Economic...",Unrated,TV 1,2009-03-07,19:00:00,12812,https://cdn.etv.org.nz/etv/TV_One_1900_0703200...,51865839
0,etv/TV_One_1900_14032009_20090314193119768qtst...,Country Calendar - Series 2009 - Episode 4,'High-flying Fur' What's as fine as merino woo...,00:29:56,Documentary,"Agriculture & Horticulture, Biology, Business,...",Unrated,TV 1,2009-03-14,19:00:00,12939,https://cdn.etv.org.nz/etv/TV_One_1900_1403200...,154209978
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,etv/hyundai_country_cale_tv1_20240414_1900,Hyundai Country Calendar - Season 2024 - 14 Ap...,NaN,00:30:33,"Documentary, Reality","Agriculture & Horticulture, Business, Careers,...",G,TV 1,2024-04-14,19:00:00,518923,https://cdn.etv.org.nz/etv/hyundai_country_cal...,353531481
0,etv/hyundai_country_cale_tv1_20240421_1900,Hyundai Country Calendar - Season 2024 - 21 Ap...,NaN,00:30:21,"Documentary, Reality","Agriculture & Horticulture, Business, Careers,...",G,TV 1,2024-04-21,19:00:00,518922,https://cdn.etv.org.nz/etv/hyundai_country_cal...,344594242
0,etv/hyundai_country_cale_tv1_20240428_1900,Hyundai Country Calendar - Season 2024 - 28 Ap...,NaN,00:30:10,"Documentary, Reality","Agriculture & Horticulture, Business, Careers,...",G,TV 1,2024-04-28,19:00:00,518972,https://cdn.etv.org.nz/etv/hyundai_country_cal...,338647527
0,etv/hyundai_country_cale_tv1_20240505_1900,Hyundai Country Calendar - Season 2024 - 5 May...,NaN,00:30:45,"Documentary, Reality","Agriculture & Horticulture, Business, Careers,...",G,TV 1,2024-05-05,19:00:00,519110,https://cdn.etv.org.nz/etv/hyundai_country_cal...,352367660


In [20]:
pd.to_timedelta(all_meta.Duration).sum()

Timedelta('10 days 23:46:30')

In [25]:
# GB
all_meta.size_bytes.astype(int).sum() / 1e9

153.993704597

In [26]:
all_meta.sort_values("Recording Date").to_csv("country_calendar_metadata.csv", index=False)